In [3]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency, fisher_exact
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Set file path for the data
file_path = r"C:\Users\Aru\Downloads\archive\marriage_data_india.csv"

# Check if file exists and load data
try:
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        print(f"Successfully loaded data from {file_path}")
        print(f"Number of records: {len(df)}")
        print("\nColumns in the dataset:")
        print(df.columns.tolist())
        print("\nFirst few rows:")
        print(df.head())
    else:
        print(f"File not found at: {file_path}")
        print("Creating sample data for demonstration instead.")
        
        # Create sample data for demonstration
        np.random.seed(42)
        n = 500
        
        # Create sample data that mimics expected structure
        data = {
            'Marriage_Type': np.random.choice(['Love', 'Arranged'], size=n, p=[0.4, 0.6]),
            'Caste_Match': np.random.choice(['Yes', 'No'], size=n, p=[0.7, 0.3]),
            'Urban_Rural': np.random.choice(['Urban', 'Rural'], size=n, p=[0.6, 0.4]),
            'Divorce_Status': np.random.choice(['Yes', 'No'], size=n, p=[0.2, 0.8]),
            'Spouse_Working': np.random.choice(['Yes', 'No'], size=n, p=[0.5, 0.5]),
            'Inter-Caste': np.random.choice(['Yes', 'No'], size=n, p=[0.25, 0.75]),
            'Inter-Religion': np.random.choice(['Yes', 'No'], size=n, p=[0.15, 0.85])
        }
        
        # Create the DataFrame
        df = pd.DataFrame(data)
        
        print("\nNote: This is sample data. Please verify the file path and try again.")
        print("Possible issues:")
        print("1. The file path may be incorrect.")
        print("2. The file may be named differently.")
        print("3. The file may be in a different location.")
        print("\nSuggested actions:")
        print("1. Check that the file exists at the specified path.")
        print("2. Make sure the file name is exactly 'marriage_data_india.csv'.")
        print("3. Try using the full absolute path to the file.")
        print("4. If using Jupyter notebook, place the file in the same directory as the notebook.")
        
except Exception as e:
    print(f"Error loading the data: {e}")
    
    # Create sample data for demonstration
    np.random.seed(42)
    n = 500
    
    # Create sample data
    data = {
        'Marriage_Type': np.random.choice(['Love', 'Arranged'], size=n, p=[0.4, 0.6]),
        'Caste_Match': np.random.choice(['Yes', 'No'], size=n, p=[0.7, 0.3]),
        'Urban_Rural': np.random.choice(['Urban', 'Rural'], size=n, p=[0.6, 0.4]),
        'Divorce_Status': np.random.choice(['Yes', 'No'], size=n, p=[0.2, 0.8]),
        'Spouse_Working': np.random.choice(['Yes', 'No'], size=n, p=[0.5, 0.5]),
        'Inter-Caste': np.random.choice(['Yes', 'No'], size=n, p=[0.25, 0.75]),
        'Inter-Religion': np.random.choice(['Yes', 'No'], size=n, p=[0.15, 0.85])
    }
    
    # Create the DataFrame
    df = pd.DataFrame(data)

# Ensure column names are in the expected format
# If needed, rename columns to match our expected format
expected_columns = ['Marriage_Type', 'Caste_Match', 'Urban_Rural', 'Divorce_Status', 
                    'Spouse_Working', 'Inter-Caste', 'Inter-Religion']

# Check if all expected columns exist (case insensitive)
for expected_col in expected_columns:
    if expected_col not in df.columns:
        # Try to find a case-insensitive match
        matches = [col for col in df.columns if col.lower() == expected_col.lower()]
        if matches:
            df.rename(columns={matches[0]: expected_col}, inplace=True)
        else:
            print(f"Warning: Column '{expected_col}' not found in the dataset.")

# Introduction to the statistical approach
print("\n\nChi-Square Test Analysis for Marriage Type Associations")
print("="*60)
print("\nThis analysis examines the association between Marriage Type (Love/Arranged)")
print("and several categorical variables using chi-square tests of independence.")
print("\nFor each variable, we will:")
print("1. Create a contingency table")
print("2. Check chi-square test assumptions")
print("3. Perform appropriate statistical test (chi-square or Fisher's exact test)")
print("4. Report and interpret results\n")

# Function to perform chi-square test or Fisher's exact test with assumption checking
def perform_association_test(df, var, primary_var='Marriage_Type'):
    """
    Perform chi-square test or Fisher's exact test based on assumptions.
    
    Parameters:
    df (DataFrame): The data
    var (str): Variable to test for association with primary_var
    primary_var (str): Primary variable (default 'Marriage_Type')
    
    Returns:
    dict: Results of the test
    """
    # Skip if the column doesn't exist
    if var not in df.columns or primary_var not in df.columns:
        print(f"\nSkipping test for {var} as it or {primary_var} is not in the dataset.")
        return None
    
    # Skip if the column has all missing values
    if df[var].isna().all() or df[primary_var].isna().all():
        print(f"\nSkipping test for {var} as it or {primary_var} contains all missing values.")
        return None
    
    # Create a copy with non-missing values for this test
    test_df = df[[primary_var, var]].dropna()
    
    # Skip if too few data points after dropping missing values
    if len(test_df) < 10:
        print(f"\nSkipping test for {var} as there are too few non-missing values.")
        return None
    
    # Create contingency table
    try:
        cont_table = pd.crosstab(test_df[primary_var], test_df[var])
        print(f"\nContingency Table for {primary_var} vs {var}:")
        print(cont_table)
        
        # Check if contingency table dimensions are valid (at least 2x2)
        if cont_table.shape[0] < 2 or cont_table.shape[1] < 2:
            print(f"\nSkipping test for {var} as the contingency table doesn't have at least two categories for each variable.")
            return None
        
        # Check assumptions for chi-square test
        # 1. Independence assumption is assumed from research design
        # 2. Check if all expected frequencies are at least 5
        chi2, p, dof, expected = chi2_contingency(cont_table)
        
        # Print expected frequencies
        print(f"\nExpected Frequencies:")
        expected_df = pd.DataFrame(
            expected, 
            index=cont_table.index, 
            columns=cont_table.columns
        )
        print(expected_df)
        
        # Check if any expected frequency is less than 5
        assumption_met = np.all(expected >= 5)
        print(f"\nAll expected frequencies >= 5: {assumption_met}")
        
        # Perform appropriate test
        if assumption_met:
            # Chi-square test
            test_type = "Chi-square test"
            stat, p_value, _, _ = chi2_contingency(cont_table)
            test_stat = stat
        else:
            # Fisher's exact test for 2x2 tables
            if cont_table.shape == (2, 2):
                test_type = "Fisher's exact test"
                odds_ratio, p_value = fisher_exact(cont_table)
                test_stat = odds_ratio
            else:
                # For larger tables where assumptions aren't met, report chi-square with warning
                test_type = "Chi-square test (Warning: assumptions not met)"
                stat, p_value, _, _ = chi2_contingency(cont_table)
                test_stat = stat
        
        # Prepare results
        results = {
            'variable': var,
            'test_type': test_type,
            'test_statistic': test_stat,
            'p_value': p_value,
            'significant': p_value < 0.05,
            'assumption_met': assumption_met,
            'contingency_table': cont_table,
            'expected_frequencies': expected_df
        }
        
        return results
    
    except Exception as e:
        print(f"\nError performing test for {var}: {e}")
        return None

# Variables to test
variables = ['Caste_Match', 'Urban_Rural', 'Divorce_Status', 
            'Spouse_Working', 'Inter-Caste', 'Inter-Religion']

# Perform the tests and collect results
results = []
for var in variables:
    print("\n" + "="*60)
    print(f"Testing association between Marriage_Type and {var}")
    result = perform_association_test(df, var)
    
    if result is None:
        print(f"Could not perform test for {var}.")
        continue
    
    # Print results
    print(f"\nResults of {result['test_type']}:")
    if result['test_type'] == "Fisher's exact test":
        print(f"Odds Ratio: {result['test_statistic']:.4f}")
    else:
        print(f"Chi-square statistic: {result['test_statistic']:.4f}")
    print(f"p-value: {result['p_value']:.4f}")
    print(f"Statistically significant (α=0.05): {result['significant']}")
    
    # Interpretation
    if result['significant']:
        print(f"\nInterpretation: There is a statistically significant association between")
        print(f"Marriage_Type and {var} (p = {result['p_value']:.4f}).")
        
        # Add effect size calculation (Cramer's V) for chi-square
        if "Chi-square" in result['test_type']:
            n = result['contingency_table'].sum().sum()
            min_dim = min(result['contingency_table'].shape) - 1
            cramer_v = np.sqrt(result['test_statistic'] / (n * min_dim))
            print(f"Effect size (Cramer's V): {cramer_v:.4f}")
            
            # Interpret effect size
            if cramer_v < 0.1:
                effect = "negligible"
            elif cramer_v < 0.3:
                effect = "small"
            elif cramer_v < 0.5:
                effect = "medium"
            else:
                effect = "large"
            print(f"This represents a {effect} effect size.")
    else:
        print(f"\nInterpretation: There is no statistically significant association between")
        print(f"Marriage_Type and {var} (p = {result['p_value']:.4f}).")
    
    results.append(result)

# Create a summary table if we have results
if results:
    summary_data = []
    for result in results:
        # Add effect size calculation for summary table
        if "Chi-square" in result['test_type']:
            n = result['contingency_table'].sum().sum()
            min_dim = min(result['contingency_table'].shape) - 1
            cramer_v = np.sqrt(result['test_statistic'] / (n * min_dim))
            effect_size = f"{cramer_v:.4f}"
        else:
            effect_size = "N/A"  # For Fisher's exact test
            
        summary_data.append({
            'Variable': result['variable'],
            'Test Type': result['test_type'],
            'Test Statistic': f"{result['test_statistic']:.4f}",
            'p-value': f"{result['p_value']:.4f}",
            'Significant': 'Yes' if result['significant'] else 'No',
            'Assumptions Met': 'Yes' if result['assumption_met'] else 'No',
            'Effect Size': effect_size
        })

    summary_df = pd.DataFrame(summary_data)
    print("\n" + "="*60)
    print("\nSummary of Results:")
    print(summary_df.to_string(index=False))

    # Visualization of results
    if len(results) > 0:
        num_rows = (len(results) + 1) // 2  # Calculate rows needed
        plt.figure(figsize=(12, 3 * num_rows))
        
        for i, result in enumerate(results):
            plt.subplot(num_rows, 2, i+1)
            
            # Create a heatmap of the contingency table
            sns.heatmap(
                result['contingency_table'], 
                annot=True, 
                fmt='d', 
                cmap='YlGnBu',
                cbar=False
            )
            
            # Add a more informative title with p-value and significance
            sig_marker = "*" if result['significant'] else "ns"
            plt.title(f"Marriage_Type vs {result['variable']}\np = {result['p_value']:.4f} {sig_marker}")
            plt.tight_layout()

        try:
            plt.savefig('marriage_type_associations.png')
            print("\nAnalysis complete. A visualization of the contingency tables has been saved as 'marriage_type_associations.png'.")
        except Exception as e:
            print(f"\nCould not save visualization: {e}")
            
        plt.close()
else:
    print("\nNo valid results to summarize. Please check your data and file path.")
    print("If using your own data, ensure the column names match the expected format:")
    print(", ".join(expected_columns))

Successfully loaded data from C:\Users\Aru\Downloads\archive\marriage_data_india.csv
Number of records: 10000

Columns in the dataset:
['ID', 'Marriage_Type', 'Age_at_Marriage', 'Gender', 'Education_Level', 'Caste_Match', 'Religion', 'Parental_Approval', 'Urban_Rural', 'Dowry_Exchanged', 'Marital_Satisfaction', 'Divorce_Status', 'Children_Count', 'Income_Level', 'Years_Since_Marriage', 'Spouse_Working', 'Inter-Caste', 'Inter-Religion']

First few rows:
   ID Marriage_Type  Age_at_Marriage  Gender Education_Level Caste_Match  \
0   1          Love               23    Male        Graduate   Different   
1   2          Love               28  Female          School        Same   
2   3      Arranged               39    Male    Postgraduate        Same   
3   4      Arranged               26  Female          School   Different   
4   5          Love               32  Female        Graduate        Same   

  Religion Parental_Approval Urban_Rural Dowry_Exchanged Marital_Satisfaction  \
0    

The above binary variables were tested with Marriage_Type. Assumptions were tested first in order to determine if a non-parametric test was needed to be performed. 
According to these results, the only variable that was associated with Marriage_Type (arranged v love) was Spouse_Working (whether one's spouse was working or not); however, this effect size was small. 

In [10]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency, fisher_exact
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Set file path for the data
file_path = r"C:\Users\Aru\Downloads\archive\marriage_data_india.csv"

# Function to check if column exists in dataset
def check_column(df, column_name):
    if column_name not in df.columns:
        # Try to find a case-insensitive match
        matches = [col for col in df.columns if col.lower() == column_name.lower()]
        if matches:
            return matches[0]
        else:
            return None
    return column_name

# Function to perform chi-square test or Fisher's exact test
def perform_chi_square_test(df, var1, var2):
    """
    Perform chi-square test or Fisher's exact test based on assumptions.
    
    Parameters:
    df (DataFrame): The data
    var1 (str): First variable name
    var2 (str): Second variable name
    
    Returns:
    dict: Results of the test
    """
    # Get correct column names
    var1_col = check_column(df, var1)
    var2_col = check_column(df, var2)
    
    # Skip if any column doesn't exist
    if not var1_col or not var2_col:
        print(f"\nSkipping test: Could not find columns {var1} or {var2}")
        return None
    
    # Create a copy with only non-missing values for this test
    test_df = df[[var1_col, var2_col]].dropna()
    
    # Skip if too few data points after dropping missing values
    if len(test_df) < 10:
        print(f"\nSkipping test: Too few non-missing values for {var1_col} vs {var2_col}")
        return None
        
    # For continuous variables, bin them first
    if pd.api.types.is_numeric_dtype(test_df[var2_col]):
        if test_df[var2_col].nunique() > 10:  # Arbitrary cutoff for binning
            try:
                # Try to bin the variable into quartiles
                test_df[f"{var2_col}_binned"] = pd.qcut(test_df[var2_col], q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'], duplicates='drop')
                var2_col = f"{var2_col}_binned"
                print(f"Binned continuous variable {var2} into quartiles.")
            except Exception as e:
                print(f"Could not bin variable {var2}: {e}")
                # If binning fails, let's try equal-width bins
                try:
                    test_df[f"{var2_col}_binned"] = pd.cut(test_df[var2_col], bins=4, labels=['Bin1', 'Bin2', 'Bin3', 'Bin4'])
                    var2_col = f"{var2_col}_binned"
                    print(f"Binned continuous variable {var2} into equal-width bins.")
                except:
                    print(f"Could not bin variable {var2} with equal-width bins either. Using original values.")
    
    # Create contingency table
    try:
        cont_table = pd.crosstab(test_df[var1_col], test_df[var2_col])
        print(f"\nContingency Table for {var1_col} vs {var2_col}:")
        print(cont_table)
        
        # Check if contingency table dimensions are valid (at least 2x2)
        if cont_table.shape[0] < 2 or cont_table.shape[1] < 2:
            print(f"\nSkipping test: The contingency table doesn't have at least two categories for each variable.")
            return None
        
        # Check assumptions for chi-square test
        chi2, p, dof, expected = chi2_contingency(cont_table)
        
        # Print expected frequencies
        print(f"\nExpected Frequencies:")
        expected_df = pd.DataFrame(
            expected, 
            index=cont_table.index, 
            columns=cont_table.columns
        )
        print(expected_df)
        
        # Check if any expected frequency is less than 5
        assumption_met = np.all(expected >= 5)
        print(f"\nAll expected frequencies >= 5: {assumption_met}")
        
        # Perform appropriate test
        if assumption_met:
            # Chi-square test
            test_type = "Chi-square test"
            stat, p_value, _, _ = chi2_contingency(cont_table)
            test_stat = stat
        else:
            # Fisher's exact test for 2x2 tables
            if cont_table.shape == (2, 2):
                test_type = "Fisher's exact test"
                odds_ratio, p_value = fisher_exact(cont_table)
                test_stat = odds_ratio
            else:
                # For larger tables where assumptions aren't met, report chi-square with warning
                test_type = "Chi-square test (Warning: assumptions not met)"
                stat, p_value, _, _ = chi2_contingency(cont_table)
                test_stat = stat
        
        # Calculate Cramer's V for effect size (for chi-square)
        if "Chi-square" in test_type:
            n = cont_table.sum().sum()
            min_dim = min(cont_table.shape) - 1
            cramers_v = np.sqrt(stat / (n * min_dim))
        else:
            cramers_v = None
        
        # Prepare results
        results = {
            'variable': var2_col,
            'test_type': test_type,
            'test_statistic': test_stat,
            'p_value': p_value,
            'significant': p_value < 0.05,
            'assumption_met': assumption_met,
            'contingency_table': cont_table,
            'expected_frequencies': expected_df,
            'cramers_v': cramers_v
        }
        
        return results
    
    except Exception as e:
        print(f"\nError performing test for {var1_col} vs {var2_col}: {e}")
        return None

try:
    # Check if file exists
    if os.path.exists(file_path):
        # Load data
        df = pd.read_csv(file_path)
        print(f"Successfully loaded data from {file_path}")
        print(f"Number of records: {len(df)}")
        print("\nColumns in the dataset:")
        print(df.columns.tolist())
        print("\nFirst few rows:")
        print(df.head())
    else:
        print(f"File not found at: {file_path}")
        print("Creating sample data for demonstration instead.")
        
        # Create sample data for demonstration
        np.random.seed(42)
        n = 500
        
        # Create sample data
        education_levels = ['Primary', 'Secondary', 'Graduate', 'Post-Graduate']
        religion = ['Hindu', 'Muslim', 'Christian', 'Sikh', 'Other']
        income_levels = ['Low', 'Medium', 'High', 'Very High']
        
        marriage_type = np.random.choice(['Love', 'Arranged'], size=n, p=[0.4, 0.6])
        
        # Education_Level - slightly higher education in love marriages
        p_edu_high = np.where(marriage_type == 'Love', [0.1, 0.3, 0.4, 0.2], [0.2, 0.4, 0.3, 0.1])
        education_level = [np.random.choice(education_levels, p=p) for p in p_edu_high]
        
        # Caste_Match - more likely in arranged marriages
        p_caste_match = np.where(marriage_type == 'Arranged', 0.85, 0.45)
        caste_match = np.random.binomial(1, p_caste_match, n) == 1
        caste_match = np.where(caste_match, 'Yes', 'No')
        
        # Religion - no strong pattern
        religion_choice = np.random.choice(religion, size=n)
        
        # Parental_Approval - higher in arranged marriages
        p_approval = np.where(marriage_type == 'Arranged', 0.9, 0.6)
        parental_approval = np.random.binomial(1, p_approval, n) == 1
        parental_approval = np.where(parental_approval, 'Yes', 'No')
        
        # Dowry_Exchanged - more common in arranged marriages
        p_dowry = np.where(marriage_type == 'Arranged', 0.7, 0.3)
        dowry_exchanged = np.random.binomial(1, p_dowry, n) == 1
        dowry_exchanged = np.where(dowry_exchanged, 'Yes', 'No')
        
        # Marital_Satisfaction - slightly higher in love marriages
        marital_satisfaction = np.random.choice(['Low', 'Medium', 'High'], size=n, 
                                             p=np.where(marriage_type == 'Love', 
                                                        [0.2, 0.3, 0.5], 
                                                        [0.3, 0.4, 0.3]))
        
        # Children_Count - no strong pattern
        children_count = np.random.poisson(2, n)
        
        # Income_Level - no strong pattern
        income_level = np.random.choice(income_levels, size=n)
        
        # Create the DataFrame
        df = pd.DataFrame({
            'Marriage_Type': marriage_type,
            'Education_Level': education_level,
            'Caste_Match': caste_match,
            'Religion': religion_choice,
            'Parental_Approval': parental_approval,
            'Dowry_Exchanged': dowry_exchanged,
            'Marital_Satisfaction': marital_satisfaction,
            'Children_Count': children_count,
            'Income_Level': income_level
        })
        
        print("\nNote: This is sample data. Please verify the file path and try again.")
        
except Exception as e:
    print(f"Error loading the data: {e}")
    # Similar sample data creation as above would go here if this exception is triggered

# Get the correct column name for Marriage_Type
marriage_type_col = check_column(df, 'Marriage_Type')
if not marriage_type_col:
    print("Error: Marriage_Type column not found in the dataset.")
    print("Available columns:", df.columns.tolist())
    exit()

# Print unique values of Marriage_Type to verify
print(f"\nUnique values in {marriage_type_col}: {df[marriage_type_col].unique()}")

# List of variables to test against Marriage_Type
variables_to_test = [
    'Education_Level', 
    'Caste_Match', 
    'Religion', 
    'Parental_Approval', 
    'Dowry_Exchanged', 
    'Marital_Satisfaction', 
    'Children_Count', 
    'Income_Level'
]

# Perform chi-square tests for Marriage_Type vs each specified variable
results = []

print("\n\n" + "="*70)
print("CHI-SQUARE ANALYSIS: Marriage_Type vs Specified Variables")
print("="*70)

for var in variables_to_test:
    print("\n" + "-"*70)
    print(f"Testing association between {marriage_type_col} and {var}")
    print("-"*70)
    
    result = perform_chi_square_test(df, marriage_type_col, var)
    
    if result:
        # Print results
        print(f"\nResults of {result['test_type']}:")
        if result['test_type'] == "Fisher's exact test":
            print(f"Odds Ratio: {result['test_statistic']:.4f}")
        else:
            print(f"Chi-square statistic: {result['test_statistic']:.4f}")
        print(f"p-value: {result['p_value']:.4f}")
        print(f"Statistically significant (α=0.05): {result['significant']}")
        
        # Print effect size for chi-square tests
        if result['cramers_v'] is not None:
            print(f"Effect size (Cramer's V): {result['cramers_v']:.4f}")
            
            # Interpret effect size
            if result['cramers_v'] < 0.1:
                effect = "negligible"
            elif result['cramers_v'] < 0.3:
                effect = "small"
            elif result['cramers_v'] < 0.5:
                effect = "moderate"
            else:
                effect = "large"
            print(f"This represents a {effect} effect size.")
        
        # Interpretation
        if result['significant']:
            print(f"\nInterpretation: There is a statistically significant association between")
            print(f"{marriage_type_col} and {result['variable']} (p = {result['p_value']:.4f}).")
            
            # Analyze the patterns in contingency table
            table = result['contingency_table']
            
            # For each type of marriage, see which category has higher percentage
            try:
                row_percentages = table.div(table.sum(axis=1), axis=0) * 100
                print("\nPercentage breakdown by marriage type:")
                print(row_percentages)
                
                # Describe the key finding
                marriage_types = table.index.tolist()
                
                for m_type in marriage_types:
                    max_cat = row_percentages.loc[m_type].idxmax()
                    max_pct = row_percentages.loc[m_type, max_cat]
                    print(f"- {m_type} marriages have higher percentage of '{max_cat}' ({max_pct:.1f}%)")
            except Exception as e:
                print(f"Could not calculate percentages: {e}")
                
        else:
            print(f"\nInterpretation: There is no statistically significant association between")
            print(f"{marriage_type_col} and {result['variable']} (p = {result['p_value']:.4f}).")
        
        results.append(result)

# Create a summary table
if results:
    summary_data = []
    for result in results:
        summary_data.append({
            'Variable': result['variable'],
            'Test Type': result['test_type'],
            'Test Statistic': f"{result['test_statistic']:.4f}",
            'p-value': f"{result['p_value']:.4f}",
            'Significant': 'Yes' if result['significant'] else 'No',
            'Effect Size': f"{result['cramers_v']:.4f}" if result['cramers_v'] is not None else 'N/A'
        })

    summary_df = pd.DataFrame(summary_data)
    # Sort by p-value to highlight most significant associations first
    summary_df = summary_df.sort_values('p-value')
    
    print("\n" + "="*70)
    print("\nSUMMARY OF CHI-SQUARE TESTS (sorted by p-value)")
    print("="*70)
    print(summary_df.to_string(index=False))

    # Create visualizations
    if len(results) > 0:
        # Create a grid of plots
        num_rows = (len(results) + 1) // 2
        fig, axes = plt.subplots(num_rows, 2, figsize=(15, 5 * num_rows))
        axes = axes.flatten() if num_rows > 1 else [axes] if isinstance(axes, np.ndarray) else [axes]
        
        for i, result in enumerate(results):
            if i < len(axes):  # Ensure we don't go out of bounds
                ax = axes[i]
                
                # Create a heatmap of the contingency table
                cont_table = result['contingency_table']
                
                try:
                    # Calculate percentages by row
                    normalized_table = cont_table.div(cont_table.sum(axis=1), axis=0) * 100
                    
                    # Plot the heatmap
                    sns.heatmap(
                        normalized_table,
                        annot=True, 
                        fmt='.1f', 
                        cmap='YlGnBu',
                        cbar_kws={'label': 'Percentage (%)'},
                        ax=ax
                    )
                    
                    # Add title with p-value and significance
                    sig_marker = "*" if result['significant'] else "ns"
                    effect_size = f", V={result['cramers_v']:.2f}" if result['cramers_v'] is not None else ""
                    ax.set_title(f"{marriage_type_col} vs {result['variable']}\np = {result['p_value']:.4f} {sig_marker}{effect_size}")
                except Exception as e:
                    ax.text(0.5, 0.5, f"Could not create heatmap: {e}", 
                            ha='center', va='center', transform=ax.transAxes)
        
        # Hide any unused subplots
        for j in range(i+1, len(axes)):
            axes[j].axis('off')
        
        plt.tight_layout()
        
        try:
            plt.savefig('marriage_type_associations.png')
            print("\nVisualization saved as 'marriage_type_associations.png'.")
        except Exception as e:
            print(f"\nCould not save visualization: {e}")
        
        plt.close()
    
    # Print final notes
    print("\nNOTES ON INTERPRETATION:")
    print("- A significant p-value (< 0.05) indicates an association between Marriage_Type and the variable.")
    print("- Cramer's V measures the strength of association (0 to 1):")
    print("  * < 0.1: Negligible association")
    print("  * 0.1 to < 0.3: Small association")
    print("  * 0.3 to < 0.5: Moderate association")
    print("  * ≥ 0.5: Strong association")
    print("- The heat maps show the percentage distribution of responses by marriage type.")
    
    # Print specific practical interpretations based on significant findings
    significant_results = [r for r in results if r['significant']]
    if significant_results:
        print("\nKEY FINDINGS:")
        for i, result in enumerate(significant_results):
            var = result['variable']
            print(f"{i+1}. {var} is significantly associated with Marriage_Type (p={result['p_value']:.4f})")
            
            if result['cramers_v'] is not None:
                strength = "weak" if result['cramers_v'] < 0.3 else "moderate" if result['cramers_v'] < 0.5 else "strong"
                print(f"   • This is a {strength} association (Cramer's V = {result['cramers_v']:.2f})")
            
            # Try to provide some specific insights about patterns
            try:
                table = result['contingency_table']
                row_pcts = table.div(table.sum(axis=1), axis=0) * 100
                
                # For each marriage type, find the most common category
                for m_type in table.index:
                    max_cat = row_pcts.loc[m_type].idxmax()
                    max_pct = row_pcts.loc[m_type, max_cat]
                    print(f"   • {m_type} marriages are more likely to have {var}={max_cat} ({max_pct:.1f}%)")
            except:
                pass  # Skip detailed pattern analysis if it fails
    else:
        print("\nNo statistically significant associations were found.")
else:
    print("\nNo valid results to summarize. Please check your data and file path.")

Successfully loaded data from C:\Users\Aru\Downloads\archive\marriage_data_india.csv
Number of records: 10000

Columns in the dataset:
['ID', 'Marriage_Type', 'Age_at_Marriage', 'Gender', 'Education_Level', 'Caste_Match', 'Religion', 'Parental_Approval', 'Urban_Rural', 'Dowry_Exchanged', 'Marital_Satisfaction', 'Divorce_Status', 'Children_Count', 'Income_Level', 'Years_Since_Marriage', 'Spouse_Working', 'Inter-Caste', 'Inter-Religion']

First few rows:
   ID Marriage_Type  Age_at_Marriage  Gender Education_Level Caste_Match  \
0   1          Love               23    Male        Graduate   Different   
1   2          Love               28  Female          School        Same   
2   3      Arranged               39    Male    Postgraduate        Same   
3   4      Arranged               26  Female          School   Different   
4   5          Love               32  Female        Graduate        Same   

  Religion Parental_Approval Urban_Rural Dowry_Exchanged Marital_Satisfaction  \
0    

Similar to the previous analysis, the chi-Square test was used to determine the relationship between Marriage Type and categorical variables stated above. None were deemed statistically significant, with Parental Approval being the closest to significant with a negligable effect size. 

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind, mannwhitneyu, shapiro, levene
import os

# Set file path for the data
file_path = r"C:\Users\Aru\Downloads\archive\marriage_data_india.csv"

# Function to check if column exists in dataset
def check_column(df, column_name):
    if column_name not in df.columns:
        # Try to find a case-insensitive match
        matches = [col for col in df.columns if col.lower() == column_name.lower()]
        if matches:
            return matches[0]
        else:
            return None
    return column_name

# Function to perform statistical tests for continuous variables
def analyze_continuous_by_group(df, cont_var, group_var):
    """
    Analyze a continuous variable by a categorical grouping variable
    
    Parameters:
    df (DataFrame): The data
    cont_var (str): Continuous variable name
    group_var (str): Grouping variable name
    
    Returns:
    dict: Results of the analysis
    """
    # Get correct column names
    cont_var_col = check_column(df, cont_var)
    group_var_col = check_column(df, group_var)
    
    # Skip if any column doesn't exist
    if not cont_var_col or not group_var_col:
        print(f"\nSkipping analysis: Could not find columns {cont_var} or {group_var}")
        return None
    
    # Create a copy with only non-missing values for this analysis
    test_df = df[[cont_var_col, group_var_col]].dropna()
    
    # Skip if too few data points after dropping missing values
    if len(test_df) < 10:
        print(f"\nSkipping analysis: Too few non-missing values for {cont_var_col} vs {group_var_col}")
        return None
    
    # Check if continuous variable is actually numeric
    if not pd.api.types.is_numeric_dtype(test_df[cont_var_col]):
        try:
            test_df[cont_var_col] = pd.to_numeric(test_df[cont_var_col], errors='coerce')
            test_df = test_df.dropna()
            print(f"\nConverted {cont_var_col} to numeric type.")
        except:
            print(f"\nCould not convert {cont_var_col} to numeric. Skipping analysis.")
            return None
    
    # Check if grouping variable has at least 2 groups
    if test_df[group_var_col].nunique() < 2:
        print(f"\nSkipping analysis: {group_var_col} has fewer than 2 groups.")
        return None
    
    # Get unique values of the grouping variable
    groups = test_df[group_var_col].unique()
    
    # Basic descriptive statistics by group
    group_stats = test_df.groupby(group_var_col)[cont_var_col].agg(['count', 'mean', 'std', 'min', 'median', 'max'])
    print(f"\nDescriptive Statistics for {cont_var_col} by {group_var_col}:")
    print(group_stats)
    
    # Test for normality for each group
    normality_results = {}
    for group in groups:
        group_data = test_df[test_df[group_var_col] == group][cont_var_col]
        if len(group_data) >= 3:  # Shapiro-Wilk requires at least 3 observations
            stat, p = shapiro(group_data)
            normality_results[group] = {
                'statistic': stat,
                'p_value': p,
                'normal': p > 0.05
            }
    
    print("\nTest for Normality (Shapiro-Wilk):")
    for group, result in normality_results.items():
        print(f"Group '{group}': p = {result['p_value']:.4f} - {'Normal' if result['normal'] else 'Not normal'}")
    
    # Decide if all groups are normally distributed
    all_normal = all(result['normal'] for result in normality_results.values()) if normality_results else False
    
    # If dealing with 2 groups, check equality of variances (for t-test)
    if len(groups) == 2 and all_normal:
        group1_data = test_df[test_df[group_var_col] == groups[0]][cont_var_col]
        group2_data = test_df[test_df[group_var_col] == groups[1]][cont_var_col]
        
        # Levene's test for homogeneity of variance
        try:
            levene_stat, levene_p = levene(group1_data, group2_data)
            equal_var = levene_p > 0.05
            print(f"\nTest for Equal Variances (Levene's test):")
            print(f"p = {levene_p:.4f} - {'Equal variances' if equal_var else 'Unequal variances'}")
        except:
            print("\nCould not perform Levene's test. Assuming unequal variances.")
            equal_var = False
    else:
        equal_var = False
    
    # Decide which test to use based on normality and number of groups
    if len(groups) == 2:  # Two groups
        group1 = groups[0]
        group2 = groups[1]
        group1_data = test_df[test_df[group_var_col] == group1][cont_var_col]
        group2_data = test_df[test_df[group_var_col] == group2][cont_var_col]
        
        # Perform both parametric and non-parametric tests
        # 1. Independent t-test (parametric)
        try:
            t_stat, t_p = ttest_ind(group1_data, group2_data, equal_var=equal_var)
            print(f"\nIndependent Samples t-test:")
            print(f"t = {t_stat:.4f}, p = {t_p:.4f}")
            print(f"Result: {'Significant difference' if t_p < 0.05 else 'No significant difference'}")
            
            # Calculate Cohen's d effect size for t-test
            if len(group1_data) > 0 and len(group2_data) > 0:
                mean1, mean2 = group1_data.mean(), group2_data.mean()
                sd1, sd2 = group1_data.std(), group2_data.std()
                n1, n2 = len(group1_data), len(group2_data)
                
                # Pooled standard deviation
                pooled_sd = np.sqrt(((n1 - 1) * sd1**2 + (n2 - 1) * sd2**2) / (n1 + n2 - 2))
                
                # Cohen's d
                if pooled_sd > 0:
                    cohens_d = abs(mean1 - mean2) / pooled_sd
                    print(f"Effect size (Cohen's d): {cohens_d:.4f}")
                    
                    # Interpret Cohen's d
                    if cohens_d < 0.2:
                        effect = "negligible"
                    elif cohens_d < 0.5:
                        effect = "small"
                    elif cohens_d < 0.8:
                        effect = "medium"
                    else:
                        effect = "large"
                    print(f"This represents a {effect} effect size.")
        except Exception as e:
            print(f"Could not perform t-test: {e}")
            t_stat, t_p = None, None
        
        # 2. Mann-Whitney U test (non-parametric)
        try:
            u_stat, u_p = mannwhitneyu(group1_data, group2_data)
            print(f"\nMann-Whitney U test:")
            print(f"U = {u_stat:.4f}, p = {u_p:.4f}")
            print(f"Result: {'Significant difference' if u_p < 0.05 else 'No significant difference'}")
            
            # Calculate effect size for Mann-Whitney U (r = Z / sqrt(N))
            n = len(group1_data) + len(group2_data)
            if n > 0:
                # Calculate Z from U
                mean_u = (len(group1_data) * len(group2_data)) / 2
                sd_u = np.sqrt((len(group1_data) * len(group2_data) * (n + 1)) / 12)
                if sd_u > 0:
                    z = (u_stat - mean_u) / sd_u
                    r = abs(z) / np.sqrt(n)
                    print(f"Effect size (r): {r:.4f}")
                    
                    # Interpret r
                    if r < 0.1:
                        effect = "negligible"
                    elif r < 0.3:
                        effect = "small"
                    elif r < 0.5:
                        effect = "medium"
                    else:
                        effect = "large"
                    print(f"This represents a {effect} effect size.")
        except Exception as e:
            print(f"Could not perform Mann-Whitney U test: {e}")
            u_stat, u_p = None, None
        
        # Decide which test to use for final results
        if all_normal:
            # Use t-test for normal data
            final_test = "t-test"
            final_stat = t_stat
            final_p = t_p
        else:
            # Use Mann-Whitney for non-normal data
            final_test = "Mann-Whitney U"
            final_stat = u_stat
            final_p = u_p
        
        # Return results
        return {
            'variable': cont_var_col,
            'group_variable': group_var_col,
            'descriptive_stats': group_stats,
            'normality': all_normal,
            'equal_variances': equal_var if 'equal_var' in locals() else None,
            'recommended_test': final_test,
            'final_statistic': final_stat,
            'final_p_value': final_p,
            'significant': final_p < 0.05 if final_p is not None else None,
            'groups': groups
        }
    else:
        # For more than 2 groups, we would use ANOVA or Kruskal-Wallis
        # Not implemented here as the question focuses on Marriage_Type which has 2 values
        print("\nAnalysis for more than 2 groups not implemented in this script.")
        return None

try:
    # Check if file exists
    if os.path.exists(file_path):
        # Load data
        df = pd.read_csv(file_path)
        print(f"Successfully loaded data from {file_path}")
        print(f"Number of records: {len(df)}")
        print("\nColumns in the dataset:")
        print(df.columns.tolist())
        print("\nFirst few rows:")
        print(df.head())
    else:
        print(f"File not found at: {file_path}")
        print("Creating sample data for demonstration instead.")
        
        # Create sample data for demonstration
        np.random.seed(42)
        n = 500
        
        # Create sample data with meaningful patterns
        marriage_type = np.random.choice(['Love', 'Arranged'], size=n, p=[0.4, 0.6])
        
        # Age_at_Marriage - slightly lower for love marriages on average
        age_at_marriage = np.where(marriage_type == 'Love',
                      np.random.normal(24, 4, n),  # Love marriages: mean age 24
                      np.random.normal(27, 4, n))  # Arranged marriages: mean age 27
        
        # Years_Since_Marriage - slightly shorter for love marriages on average
        years_since_marriage = np.where(marriage_type == 'Love',
                                       np.random.gamma(4, 1, n),  # Love marriages: shorter duration
                                       np.random.gamma(6, 1, n))  # Arranged marriages: longer duration
        
        # Create DataFrame
        df = pd.DataFrame({
            'Marriage_Type': marriage_type,
            'Age_at_Marriage': age_at_marriage,
            'Years_Since_Marriage': years_since_marriage
        })
        
        print("\nNote: This is sample data. Please verify the file path and try again.")
        
except Exception as e:
    print(f"Error loading the data: {e}")
    # Create sample data as a fallback
    np.random.seed(42)
    n = 500
    
    # Create basic sample data
    marriage_type = np.random.choice(['Love', 'Arranged'], size=n, p=[0.4, 0.6])
    age_at_marriage = np.random.normal(25, 5, n)
    years_since_marriage = np.random.gamma(5, 1, n)
    
    # Create DataFrame
    df = pd.DataFrame({
        'Marriage_Type': marriage_type,
        'Age_at_Marriage': age_at_marriage,
        'Years_Since_Marriage': years_since_marriage
    })

# Get the correct column name for Marriage_Type
marriage_type_col = check_column(df, 'Marriage_Type')
if not marriage_type_col:
    print("Error: Marriage_Type column not found in the dataset.")
    print("Available columns:", df.columns.tolist())
    exit()

# Print unique values of Marriage_Type to verify
print(f"\nUnique values in {marriage_type_col}: {df[marriage_type_col].unique()}")

# Continuous variables to test against Marriage_Type
continuous_vars = ['Age_at_Marriage', 'Years_Since_Marriage']

# Analyze each continuous variable
results = []

print("\n\n" + "="*70)
print("ANALYSIS OF CONTINUOUS VARIABLES BY MARRIAGE TYPE")
print("="*70)

for var in continuous_vars:
    print("\n" + "-"*70)
    print(f"Analyzing {var} by {marriage_type_col}")
    print("-"*70)
    
    result = analyze_continuous_by_group(df, var, marriage_type_col)
    
    if result:
        # Add to results list
        results.append(result)
        
        # Create visualizations
        print("\nCreating visualizations...")
        
        # 1. Box plot
        plt.figure(figsize=(10, 6))
        ax = sns.boxplot(x=marriage_type_col, y=var, data=df)
        ax.set_title(f"{var} by {marriage_type_col}")
        
        # Add statistical annotation
        if result['final_p_value'] is not None:
            sig_text = f"p = {result['final_p_value']:.4f}"
            if result['significant']:
                sig_text += " *"
            plt.text(0.5, 0.01, sig_text, 
                     horizontalalignment='center',
                     verticalalignment='bottom',
                     transform=ax.transAxes,
                     fontsize=12)
        
        try:
            plt.savefig(f"{var}_by_{marriage_type_col}_boxplot.png")
            print(f"Boxplot saved as '{var}_by_{marriage_type_col}_boxplot.png'.")
        except Exception as e:
            print(f"Could not save boxplot: {e}")
        
        plt.close()
        
        # 2. Density plot
        plt.figure(figsize=(10, 6))
        for group in result['groups']:
            group_data = df[df[marriage_type_col] == group][var].dropna()
            sns.kdeplot(group_data, label=group)
        
        plt.title(f"Distribution of {var} by {marriage_type_col}")
        plt.xlabel(var)
        plt.ylabel("Density")
        plt.legend()
        
        try:
            plt.savefig(f"{var}_by_{marriage_type_col}_density.png")
            print(f"Density plot saved as '{var}_by_{marriage_type_col}_density.png'.")
        except Exception as e:
            print(f"Could not save density plot: {e}")
            
        plt.close()

# Create a summary table
if results:
    summary_data = []
    for result in results:
        if result['final_p_value'] is not None:
            summary_data.append({
                'Variable': result['variable'],
                'Test Used': result['recommended_test'],
                'Test Statistic': f"{result['final_statistic']:.4f}" if result['final_statistic'] is not None else 'N/A',
                'p-value': f"{result['final_p_value']:.4f}" if result['final_p_value'] is not None else 'N/A',
                'Significant': 'Yes' if result['significant'] else 'No',
                'Normality': 'Normal' if result['normality'] else 'Non-normal'
            })

    summary_df = pd.DataFrame(summary_data)
    
    print("\n" + "="*70)
    print("\nSUMMARY OF STATISTICAL TESTS")
    print("="*70)
    print(summary_df.to_string(index=False))
    
    # Print final interpretation
    print("\nINTERPRETATION OF RESULTS:")
    for result in results:
        var = result['variable']
        if result['significant']:
            # Get group means
            stats = result['descriptive_stats']
            mean_diff = abs(stats.loc[result['groups'][0], 'mean'] - stats.loc[result['groups'][1], 'mean'])
            
            # Determine which group has higher mean
            higher_group = result['groups'][0] if stats.loc[result['groups'][0], 'mean'] > stats.loc[result['groups'][1], 'mean'] else result['groups'][1]
            
            print(f"\n- {var} is significantly different between marriage types (p={result['final_p_value']:.4f}).")
            print(f"  * {higher_group} marriages have higher {var} values on average.")
            print(f"  * The mean difference is {mean_diff:.2f} units.")
            print(f"  * {result['recommended_test']} was used since the data is {'normally' if result['normality'] else 'not normally'} distributed.")
        else:
            print(f"\n- No significant difference in {var} between marriage types (p={result['final_p_value']:.4f}).")
            print(f"  * {result['recommended_test']} was used since the data is {'normally' if result['normality'] else 'not normally'} distributed.")
else:
    print("\nNo valid results to summarize. Please check your data and file path.")

Successfully loaded data from C:\Users\Aru\Downloads\archive\marriage_data_india.csv
Number of records: 10000

Columns in the dataset:
['ID', 'Marriage_Type', 'Age_at_Marriage', 'Gender', 'Education_Level', 'Caste_Match', 'Religion', 'Parental_Approval', 'Urban_Rural', 'Dowry_Exchanged', 'Marital_Satisfaction', 'Divorce_Status', 'Children_Count', 'Income_Level', 'Years_Since_Marriage', 'Spouse_Working', 'Inter-Caste', 'Inter-Religion']

First few rows:
   ID Marriage_Type  Age_at_Marriage  Gender Education_Level Caste_Match  \
0   1          Love               23    Male        Graduate   Different   
1   2          Love               28  Female          School        Same   
2   3      Arranged               39    Male    Postgraduate        Same   
3   4      Arranged               26  Female          School   Different   
4   5          Love               32  Female        Graduate        Same   

  Religion Parental_Approval Urban_Rural Dowry_Exchanged Marital_Satisfaction  \
0    

C:\Users\Aru\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\stats\_axis_nan_policy.py:586: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 6022.
  res = hypotest_fun_out(*samples, **kwds)


Boxplot saved as 'Age_at_Marriage_by_Marriage_Type_boxplot.png'.
Density plot saved as 'Age_at_Marriage_by_Marriage_Type_density.png'.

----------------------------------------------------------------------
Analyzing Years_Since_Marriage by Marriage_Type
----------------------------------------------------------------------

Descriptive Statistics for Years_Since_Marriage by Marriage_Type:
               count       mean        std  min  median  max
Marriage_Type                                               
Arranged        6022  25.224676  14.091917    1    25.0   49
Love            3978  24.594017  13.991730    1    25.0   49

Test for Normality (Shapiro-Wilk):
Group 'Love': p = 0.0000 - Not normal
Group 'Arranged': p = 0.0000 - Not normal

Independent Samples t-test:
t = -2.1998, p = 0.0278
Result: Significant difference
Effect size (Cohen's d): 0.0449
This represents a negligible effect size.

Mann-Whitney U test:
U = 11666819.0000, p = 0.0277
Result: Significant difference
Effect

C:\Users\Aru\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\stats\_axis_nan_policy.py:586: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 6022.
  res = hypotest_fun_out(*samples, **kwds)


Density plot saved as 'Years_Since_Marriage_by_Marriage_Type_density.png'.


SUMMARY OF STATISTICAL TESTS
            Variable      Test Used Test Statistic p-value Significant  Normality
     Age_at_Marriage Mann-Whitney U  12059695.0000  0.5616          No Non-normal
Years_Since_Marriage Mann-Whitney U  11666819.0000  0.0277         Yes Non-normal

INTERPRETATION OF RESULTS:

- No significant difference in Age_at_Marriage between marriage types (p=0.5616).
  * Mann-Whitney U was used since the data is not normally distributed.

- Years_Since_Marriage is significantly different between marriage types (p=0.0277).
  * Arranged marriages have higher Years_Since_Marriage values on average.
  * The mean difference is 0.63 units.
  * Mann-Whitney U was used since the data is not normally distributed.


As stated above, arrianged marriages have higher Years_Since_Marriage at statistically significant level, according to Mann Whitney tests which were used to substitute due to non-normal distribution. 


These results do not necessarily mean that arranged marriages are key to longevity in marriage. In fact, cultural pressures may still inhibit divorce from unhealthy relationships in India. 
As India develops, improvements in job climates, healthcare infrastructure, and attitudes, may need for this analysis to be replicated, as these can affect divorce rates. 
This analysis also suggests that arranged marriages are common across all demographics. 
However, 60% of this sample contained arranged marriages, which may have skewed results. 
It is reccomended that future samples be close to a 50/50 split between arranged and love marriages, keeping in mind that whilst the majority of Indians are arranged marriages, it may be only temporary. 